In [2]:
from qdrant_client import QdrantClient
from qdrant_client.http.models import Distance, VectorParams
from qdrant_client.http.models import PointStruct

from parsing.docx_parser import DocxParser
from parsing.chunker import Chunker
from parsing.embeddings import Embedder

In [ ]:
DOC = '***'
COLLECTION = '***'

In [4]:
parser = DocxParser()
chunker = Chunker(model_name="BAAI/bge-m3", max_tokens=512, overlap=128)
embedder = Embedder(model_name="BAAI/bge-m3", device="cuda")

In [5]:
doc = parser.parse(path=DOC)
chunks = chunker.chunk(doc)
chunk_embeddings = embedder.embed_chunks(chunks)

In [6]:
chunk = chunks[0]
emb = chunk_embeddings[0]

assert chunk.chunk_id == emb.chunk_id
assert chunk.doc_id == emb.doc_id
assert chunk.metadata["chunk_start_token"] == emb.metadata["chunk_start_token"]
assert chunk.metadata["chunk_end_token"] == emb.metadata["chunk_end_token"]


In [8]:
client = QdrantClient(url="http://localhost:6333")

client.recreate_collection(
    collection_name=COLLECTION,
    vectors_config=VectorParams(
        size=1024,
        distance=Distance.COSINE
    ),
)

/tmp/ipykernel_2576852/3855186332.py:3: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  client.recreate_collection(


True

In [9]:
points = []

for chunk in chunk_embeddings:
    point = PointStruct(
        id=chunk.chunk_id,
        vector=chunk.vector.tolist(),
        payload={
            "chunk_id": chunk.chunk_id,
            "doc_id": chunk.doc_id,
            "text": chunk.text,
            **chunk.metadata,
        },
    )
    points.append(point)

client.upsert(
    collection_name=COLLECTION,
    points=points,
)


UpdateResult(operation_id=1, status=<UpdateStatus.COMPLETED: 'completed'>)

In [10]:
from retrieval.qdrant_retriever import QdrantRetriever

retriever = QdrantRetriever(
    qdrant_client=client,
    collection_name=COLLECTION,
    embedder=embedder,
    default_top_k=5,
)

In [11]:
query = "О чём этот документ и какие в нём основные разделы?"
chunks = retriever.retrieve(query, top_k=5)

for ch in chunks:
    print(ch.score, ch.doc_id, ch.chunk_id)
    print(ch.text[:200])
    print("-" * 80)

0.41594428 eb4f6897-176f-419c-9865-2fcc101351cf 1418e65e-b137-4e27-9441-12ed3b242b62
Техническое предложение на выполнение работ по разработке “Системы контроля качества полосы ЦДС АПгкР и АПхкР” Цель проекта Разработка и внедрение Системы Контроля Качества Полосы (Далее по тексту - С
--------------------------------------------------------------------------------
0.3806231 eb4f6897-176f-419c-9865-2fcc101351cf c90f2bed-d6d2-4620-bb02-a3f97e345009
LS2D, разработанного НПП "Призма" или аналог. Этот сканер оснащён встроенной микропроцессорной системой управления и предназначен для бесконтактного измерения профиля объектов с рассеивающей поверхнос
--------------------------------------------------------------------------------
0.3802179 eb4f6897-176f-419c-9865-2fcc101351cf 52fddc70-94ea-4473-a7f0-128b1d685e45
ового потока от LED-панели. Фотодиоды фиксируют наличие и интенсивность света, прошедшего через сквозные отверстия или дефекты в контролируемом металлическом листе (Рисунок 4). Исполь

In [12]:
from openai import OpenAI

LLM_BASE_URL = "http://localhost:8000/v1"
LLM_API_KEY = "dummy"

llm_client = OpenAI(
    base_url=LLM_BASE_URL,
    api_key=LLM_API_KEY,
)

In [13]:
from retrieval.qdrant_retriever import RetrievedChunk
from typing import List

def build_prompt(query: str, chunks: List[RetrievedChunk]) -> str:
    context_parts = []
    for i, ch in enumerate(chunks, start=1):
        context_parts.append(
            f"[Фрагмент {i}]\n{ch.text}\n"
        )

    context_text = "\n".join(context_parts)

    prompt = (
        "Ты ассистент отвечающий на вопросы по документации.\n"
        "Используй только информацию из фрагментов ниже.\n"
        "Если точного ответа в фрагментах нет, так и скажи.\n\n"
        "Контекст:\n"
        f"{context_text}\n\n"
        "Вопрос пользователя:\n"
        f"{query}\n\n"
        "Ответ:"
    )
    return prompt

In [14]:
def generate_answer(prompt: str) -> str:
    resp = llm_client.chat.completions.create(
        model="Qwen/Qwen3-4B-Instruct-2507-FP8",
        messages=[
            {"role": "user", "content": prompt},
        ],
        temperature=0.1,
        max_tokens=512,
    )
    return resp.choices[0].message.content


In [15]:
def rag_answer(query: str, top_k: int = 5) -> str:
    chunks = retriever.retrieve(query, top_k=top_k)
    if not chunks:
        return "Я не нашёл релевантных ответов для ответа на вопрос"

    prompt = build_prompt(query, chunks)
    answer = generate_answer(prompt)
    return answer


In [16]:
q = "? Какие особые требования к оптической системе и её характеристикам?"
print(rag_answer(q, top_k=5))

Особые требования к оптической системе и её характеристикам, согласно представленным фрагментам, включают:

1. **Высокая чувствительность светочувствительных элементов** – использование фотодиодов (например, PIN фотодиод VBPW34S) с высокой чувствительностью, способных оперативно и точно фиксировать даже незначительные изменения освещённости, что обеспечивает точную и быструю идентификацию дефектов.

2. **Высокая пространственная разрешающая способность** – плотное расположение фотодиодов на приёмной панели позволяет точно определять местоположение и размеры сквозных отверстий, включая отверстия диаметром до 0,3 мм.

3. **Прямолинейность светового потока** – светодиоды излучают свет в определённом направлении с ограниченным углом рассеивания, что способствует эффективному прохождению света через сквозные дефекты и облегчает их обнаружение.

4. **Подавление фоновых помех** – использование узкополосных фильтров и модуляции светового сигнала позволяет фотодиодам различать полезный сигнал о